In [1]:
#Reprojecting a raster
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import numpy as np
import rasterio as rio
import pyproj

# Define target CRS
crs = "EPSG:4326"  # WGS 1984
dst_crs = pyproj.CRS(crs)

# Open the original raster image
with rio.open(r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B1.TIF") as src:
    # Calculate the transform, width, and height of the destination raster
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    
    # Copy the metadata and update for the new CRS and transform
    metadata = src.meta.copy()
    metadata.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Define the path for the reprojected output raster
    output_reprojected = os.path.join(r"G:\Manjar\github\rasterData\reprojected.TIF")
    
    # Reproject the raster and write to the new file
    with rio.open(output_reprojected, 'w', **metadata) as dst:
        for i in range(1, src.count + 1):  # Iterate through all bands
            # Read the source band
            band = src.read(i)
            
            # Reproject the band and write it to the destination
            reproject(
                source=band,
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )

In [4]:
# Check the CRS of the input raster
with rio.open(r"G:\Manjar\github\rasterData\LC08_L2SP_146040_20241018_20241028_02_T1_SR_B1.TIF") as src:
    print("Input CRS:", src.crs)

Input CRS: PROJCS["WGS 84 / UTM zone 43N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [5]:
# Check the CRS of the reprojected raster
with rio.open(output_reprojected) as dst:
    print("Reprojected CRS:", dst.crs)

Reprojected CRS: GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
